# Master Demonstration and Final Integration

Prompt for TinyTutor Notebook 0: 00_master_demo.ipynb
# TinyTutor Capstone Notebook 0: Master Demonstration and Final Integration
## Objective
Generate runnable Python code that serves as the final, end-to-end showcase for the **TinyTutor** educational system. This notebook must simulate importing and executing the fully integrated multi-agent system (`TutoringPipeline`) as defined in the modular backend structure (backend/app/pipelines/tutoring_pipeline.py). The demonstration must prove that the entire agentic architecture—including reasoning, tool use, memory influence, and evaluation—is functional.

## Implementation Requirements (Final System Demonstration)

1.  **Environment Setup:** Include standard imports (ADK, Gemini) and boilerplate for environment configuration.
2.  **Simulated Import:** Write code to simulate the import of the final `TutoringPipeline` class, which orchestrates all sub-agents (Pedagogy, Scriptwriting, Media, Safety/Evaluator).
3.  **Execution Query:** Execute the pipeline using an input query (e.g., "Explain how Photosynthesis works like I am 5"). The query must be designed to test all major components, including memory integration.
4.  **Memory Demonstration:** The execution must simulate the system successfully retrieving and applying a **previously stored personalization preference** (e.g., character theme like "pirates" or "dinosaurs" saved in Notebook 4), ensuring the `ScriptwritingAgent`'s output is customized.
5.  **Multimodal Output Synthesis (Action/Tool Use):** The final system response must synthesize the structured results from the `MediaAgent` (Notebooks 2 & 3), clearly displaying the references (URIs/paths) for:
    *   The Final Story Script (Text)
    *   Simulated Audio URI (TTS Tool Use)
    *   Simulated Video/Image Storyboard URI (Image Tool Use)
6.  **Evaluation Visibility (Agent Ops):** The final output block must clearly present the quality assessment metrics or the pass/fail status returned by the **Evaluator Agent (LLM-as-a-Judge)** (Notebook 5), ensuring the quality check is demonstrated as part of the pipeline result.
7.  **Final Value Articulation:** The output must clearly articulate the full, aggregated multimodal result to satisfy the demo requirements for the capstone project.

## Generation Prompt
"Generate the runnable Python code for the '00_master_demo.ipynb' notebook for the **TinyTutor** project. The code must simulate importing and running the complete `TutoringPipeline`. The demonstration must execute a single query that confirms: **multi-agent orchestration**, **tool usage (audio/video URIs)**, **memory retrieval (e.g., using a 'robot' theme)**, and the final **evaluation score/status** from the judge agent. Focus the code on execution and result display, reflecting the integrated system."


### `00_master_demo.ipynb` Final checklist

| Category | Requirement | Sources & Justification |
| :--- | :--- | :--- |
| **Core Concept** | Final product demonstration and submission prep. |
| **Goal** | Showcase the end-to-end functionality of the fully assembled `TinyTutorCoordinator` pipeline, proving that the system can reason, act, remember, and self-evaluate. |
| **Dependencies** | The complete system architecture defined in the `backend/` modules (NB 6). |
| **Required Tools** | Imports of the final `TutoringPipeline` and execution environment setup. |
| **Architecture** | Must reflect the final Level 3 Collaborative Multi-Agent system architecture. |
| **Good Practices** | **Value Articulation:** The output should clearly articulate the value of the agent by synthesizing text, audio reference, and image/video references, demonstrating the complex task completion in a single run. |
| **Good Practices** | **Documentation:** This notebook serves as the primary inline documentation/demo for the Kaggle submission, using Markdown cells to explain architecture and value. |